# Data preparation
Convert all raw ECG records into standardized NumPy arrays (400 Hz, 7.3 s, lead-wise z-score), build the train, validation, and external-test splits, and save each split as a compressed `.npz` file for fast loading in model notebooks.

## Notebook setup

In [1]:
# Import required libraries
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.signal import resample
from sklearn.utils import resample as resample_bal 
import ecgmentations as E
import os
import glob

from helper_code import (
    find_records, load_header, load_signals, load_label,
    reorder_signal, get_age, get_sex, get_source
)

In [16]:
# Root folder containing the three datasets
DATA_ROOT = 'data'

CODE15_DIR   = os.path.join(DATA_ROOT, 'code15_output')
SAMITROP_DIR = os.path.join(DATA_ROOT, 'samitrop_output')
PTBXL_DIR    = os.path.join(DATA_ROOT, 'ptbxl_output')

# Choose the datasets to process
PARTS_TO_PROCESS = ['exams_part6']        # e.g. ['exams_part1'] or ['exams_part0', 'exams_part1']
code15_records   = find_records(CODE15_DIR)
code15_records = [
    rid for rid in code15_records
    if any(rid.startswith(p) for p in PARTS_TO_PROCESS)
]
print(f'Will preprocess {len(code15_records):,} records from {PARTS_TO_PROCESS}')

samitrop_records = find_records(SAMITROP_DIR)
ptbxl_records    = find_records(PTBXL_DIR)

# Count WFDB records in each dataset
print(f'CODE-15 % records : {len(code15_records):,}')
print(f'SaMi-Trop records : {len(samitrop_records):,}')
print(f'PTB-XL   records  : {len(ptbxl_records):,}\n')

# Show a few file names from each folder
print(f'CODE-15 example   : {code15_records[:3]}')
print(f'SaMi-Trop example : {samitrop_records[:3]}')
print(f'PTB-XL example    : {ptbxl_records[:3]}')

Will preprocess 19,916 records from ['exams_part6']
CODE-15 % records : 19,916
SaMi-Trop records : 1,631
PTB-XL   records  : 21,799

CODE-15 example   : ['exams_part6\\1000175', 'exams_part6\\1000190', 'exams_part6\\1000201']
SaMi-Trop example : ['100726', '101191', '101193']
PTB-XL example    : ['00000\\00001_hr', '00000\\00002_hr', '00000\\00003_hr']


## Data processing

In [7]:
def preprocess_ecg(record_path, target_fs=400, target_len=2920):
    """Resample to target_fs, trim/pad to target_len, z-score each lead."""
    # Load raw signal and metadata
    signal, fields = load_signals(record_path)
    ref_leads = ['I', 'II', 'III', 'AVR', 'AVL', 'AVF',
                 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
    signal = reorder_signal(signal, fields['sig_name'], ref_leads)

    # 1) Resample if needed
    in_fs = fields['fs']
    if in_fs != target_fs:
        n_samples = int(signal.shape[0] * target_fs / in_fs)
        signal = resample(signal, n_samples, axis=0)

    # 2) Trim or zero-pad to fixed length
    if signal.shape[0] > target_len:
        signal = signal[:target_len, :]
    elif signal.shape[0] < target_len:
        pad = np.zeros((target_len - signal.shape[0], signal.shape[1]))
        signal = np.vstack([signal, pad])

    # 3) Lead-wise z-score normalization
    mu = np.nanmean(signal, axis=0)
    sigma = np.nanstd(signal, axis=0) + 1e-8
    signal = (signal - mu) / sigma

    return signal.astype(np.float32)  # shape: (2920, 12)


### Preprocess CODE-15 % records

In [17]:
# Lists for the preprocessed signals and their labels
X_code15, y_code15 = [], []

for i, rid in enumerate(code15_records, 1):
    # Build absolute path to WFDB record
    rec_path = os.path.join(CODE15_DIR, rid)

    # Apply resample → trim/pad → z-score to get (2920, 12) array
    X_code15.append(preprocess_ecg(rec_path))

    # Load binary Chagas label (True = positive, False = negative)
    y_code15.append(load_label(rec_path))

    # Progress update every 1000 records
    if i % 1000 == 0 or i == len(code15_records):
        print(f'Processed {i}/{len(code15_records)} records')

# Stack all signals into one NumPy array (N, 2920, 12) and labels into (N,)
X_code15 = np.stack(X_code15).astype(np.float32)
y_code15 = np.array(y_code15, dtype=bool)

print(f'Final CODE-15 % array shape : {X_code15.shape}')
print(f'Number of positives         : {y_code15.sum()}')

# Save to disk so future notebooks can load in seconds
np.savez_compressed('code15_part6_proc.npz', X=X_code15, y=y_code15) # rename if more code15 parts are added
print('Saved code15_part6_proc.npz')

Processed 1000/19916 records
Processed 2000/19916 records
Processed 3000/19916 records
Processed 4000/19916 records
Processed 5000/19916 records
Processed 6000/19916 records
Processed 7000/19916 records
Processed 8000/19916 records
Processed 9000/19916 records
Processed 10000/19916 records
Processed 11000/19916 records
Processed 12000/19916 records
Processed 13000/19916 records
Processed 14000/19916 records
Processed 15000/19916 records
Processed 16000/19916 records
Processed 17000/19916 records
Processed 18000/19916 records
Processed 19000/19916 records
Processed 19916/19916 records
Final CODE-15 % array shape : (19916, 2920, 12)
Number of positives         : 408
Saved code15_part6_proc.npz


### Train–validation split

In [18]:
# Load every pre-processed CODE-15 % part and concatenate
code15_parts = sorted(glob.glob('code15_part*_proc.npz')) 
X_list, y_list = [], []
for f in code15_parts:
    d = np.load(f)
    X_list.append(d['X'])
    y_list.append(d['y'])
    print('Loaded', f, d['X'].shape)
    
# Concatenate all parts into one array
X_code15 = np.concatenate(X_list)
y_code15 = np.concatenate(y_list)
print('Combined CODE-15 % shape:', X_code15.shape)

# Save the combined array
np.savez_compressed('code15_parts_0-6_proc.npz', X=X_code15, y=y_code15)
print('Saved code15_parts_0-6_proc.npz')

Loaded code15_part0_proc.npz (19901, 2920, 12)
Loaded code15_part1_proc.npz (19897, 2920, 12)
Loaded code15_part2_proc.npz (19902, 2920, 12)
Loaded code15_part3_proc.npz (19915, 2920, 12)
Loaded code15_part4_proc.npz (19919, 2920, 12)
Loaded code15_part5_proc.npz (19912, 2920, 12)
Loaded code15_part6_proc.npz (19916, 2920, 12)
Combined CODE-15 % shape: (139362, 2920, 12)
Saved code15_parts_0-6_proc.npz


In [ ]:
# Stratified 80 / 20 split for train and internal validation
X_train, X_val, y_train, y_val = train_test_split(
    X_code15, y_code15,
    test_size=0.2,
    random_state=42,
    stratify=y_code15)

print(f'Train shape : {X_train.shape}, positives: {y_train.sum()}')
print(f'Val shape   : {X_val.shape}, positives: {y_val.sum()}')

# Balance the training set: keep all positives and sample 3 negatives per positive
pos_idx = np.where(y_train)[0]
neg_idx = np.where(~y_train)[0]

neg_sample = resample_bal(
    neg_idx,
    replace=False,
    n_samples=len(pos_idx) * 3,
    random_state=42)

balanced_idx = np.hstack([pos_idx, neg_sample])

X_bal = X_train[balanced_idx]
y_bal = y_train[balanced_idx]

print(f'Balanced train shape : {X_bal.shape}, positives: {y_bal.sum()}')

# Save the training and validation sets
np.savez_compressed('train_full.npz',     X=X_train, y=y_train)   # full, imbalanced train set
np.savez_compressed('train_balanced.npz', X=X_bal, y=y_bal)       # undersampled train set
np.savez_compressed('val.npz', X=X_val, y=y_val)                  # untouched validation set
print('Saved train_full.npz, train_balanced.npz, and val.npz')

Train shape : (111489, 2920, 12), positives: 2235
Val shape   : (27873, 2920, 12), positives: 559
Balanced train shape : (8940, 2920, 12), positives: 2235
Saved train_balanced.npz and val_internal.npz


### Augment training data

To improve generalization and robustness, we apply data augmentation to the training set using the `ecgmentations` library. These transformations simulate realistic ECG artefacts commonly encountered in clinical settings. Augmentation helps the model learn to focus on disease-related signal morphology rather than irrelevant noise patterns. We apply these transformations only to the training data, validation and test sets remain untouched to ensure unbiased evaluation.

The augmentation pipeline includes the following components:

- `GaussNoise`: Adds mild to moderate broadband electronic noise to the waveform, simulating real-world electrical interference. We apply this with a 30% probability using a fixed noise variance of 0.005.
- `AmplitudeScale`: Randomly scales the amplitude of each ECG signal by ±20% to mimic differences in gain settings or skin–electrode impedance.
- `PowerlineNoise`: Injects sinusoidal noise resembling 60 Hz mains hum, which frequently contaminates ECG recordings. This is applied with 20% probability and capped at a low amplitude to preserve waveform readability.
- `TimeShift`: Applies a temporal shift of up to ±7% of the signal length (≈±200 samples) to simulate misalignment or onset jitter between leads, improving the model’s tolerance to temporal variability.
- `SinePulse`: Adds low-frequency baseline drift (0.5–2 Hz) to simulate respiration-related or motion artefacts that often appear in ambulatory ECGs.

These parameter values were chosen to maintain clinical realism while encouraging robustness. More extreme augmentations were avoided to preserve essential waveform characteristics like P-waves and QRS complexes.

In [2]:
# Load the training set, if needed
DATA_DIR = 'data/prepared'

train_full = np.load(f'{DATA_DIR}/train_balanced_parts0-6.npz')
X_train, y_train = train_full['X'], train_full['y']   # rename as needed

print('X shape:', X_train.shape)
print('y shape:', y_train.shape, 'positives:', y_train.sum())

X shape: (8940, 2920, 12)
y shape: (8940,) positives: 2235


In [ ]:
# Build augmentation pipeline
augment = E.Sequential([
    # Gaussian broadband noise
    E.GaussNoise(
        mean=0.0,
        variance=0.005,        # noisier than default (0.01 would be very loud)
        per_channel=True,
        p=0.30
    ),
    # Amplitude scaling  ±20 %  (default is ±5 %)
    E.AmplitudeScale(
        scaling_range=(-0.20, 0.20),
        p=0.30
    ),
    # 50/60 Hz mains hum   (amplitude limited to 0.2 mV)
    E.PowerlineNoise(
        ecg_frequency=400,          # our resampled ECG rate
        powerline_frequency=60,     # US mains
        amplitude_limit=0.20,
        p=0.20
    ),
    # Temporal shift  ±0.07 fraction ≈ ±200 samples at 2 920-sample trace
    E.TimeShift(
        shift_limit=0.07,
        p=0.30
    ),
    # Low-frequency baseline wander 0.5–2 Hz
    E.SinePulse(
        ecg_frequency=400,
        pulse_frequency_range=(0.5, 2.0),
        amplitude_limit=0.50,
        p=0.20
    ),
])

# Apply once to every sample in the balanced-train set
X_aug = np.empty_like(X_train)
for i, ecg in enumerate(X_train):
    X_aug[i] = augment(ecg=ecg)['ecg']     # returns dict with key 'ecg'

# Combine originals + augmented copies
X_train_aug = np.concatenate([X_train, X_aug])
y_train_aug = np.concatenate([y_train, y_train])

print(f'Augmented train shape: {X_train_aug.shape}, '
      f'positives: {y_train_aug.sum()}')

# Save for reuse
np.savez_compressed('train_bal_parts0-6_aug.npz', X=X_train_aug, y=y_train_aug)
print('Saved train_bal_parts0-6_aug.npz')

Augmented train shape: (17880, 2920, 12), positives: 4470
Saved train_full_aug.npz


### Preprocess SaMi-Trop (positives)

In [ ]:
# Convert every SaMi-Trop ECG to a (2920, 12) array
X_pos = []
for i, rid in enumerate(samitrop_records, 1):
    rec_path = os.path.join(SAMITROP_DIR, rid)
    X_pos.append(preprocess_ecg(rec_path))

    # Progress update every 500 records
    if i % 500 == 0 or i == len(samitrop_records):
        print(f'Processed {i}/{len(samitrop_records)} SaMi-Trop records')

# Stack signals and create label vector (all positives)
X_pos = np.stack(X_pos).astype(np.float32)
y_pos = np.ones(len(X_pos), dtype=bool)
print(f'SaMi-Trop array shape: {X_pos.shape}')

# Save for reuse
np.savez_compressed('samitrop_proc.npz', X=X_pos, y=y_pos)
print('Saved samitrop_proc.npz')

Processed 500/1631 SaMi-Trop records
Processed 1000/1631 SaMi-Trop records
Processed 1500/1631 SaMi-Trop records
Processed 1631/1631 SaMi-Trop records
SaMi-Trop array shape: (1631, 2920, 12)
Saved samitrop_proc.npz


### Preprocess PTB-XL (negatives)

In [22]:
BATCH_SIZE = 1000        # number of ECGs per temporary file
batch_X, temp_files = [], []
batch_id = 0

for i, rid in enumerate(ptbxl_records, 1):
    rec_path = os.path.join(PTBXL_DIR, rid)
    batch_X.append(preprocess_ecg(rec_path))       # (2920, 12) float32

    # Progress update every 1 000 records
    if i % 1000 == 0 or i == len(ptbxl_records):
        print(f'Processed {i}/{len(ptbxl_records)} PTB-XL records')

    # When the batch is full or at the last record, save and reset
    if len(batch_X) == BATCH_SIZE or i == len(ptbxl_records):
        batch_X = np.stack(batch_X).astype(np.float32)
        fname = f'ptbxl_batch_{batch_id}.npy'
        np.save(fname, batch_X)
        temp_files.append(fname)
        print(f'Saved {fname}  shape {batch_X.shape}')
        batch_X = []
        batch_id += 1

# Concatenate all temporary batches with minimal RAM usage
X_neg = np.concatenate([np.load(f) for f in temp_files])
y_neg = np.zeros(len(X_neg), dtype=bool)
print(f'Final PTB-XL array shape: {X_neg.shape}')

# Save the full processed PTB-XL set
np.savez_compressed('ptbxl_proc.npz', X=X_neg, y=y_neg)
print('Saved ptbxl_proc.npz')

# Remove temporary files
for f in temp_files:
    os.remove(f)


Processed 1000/21799 PTB-XL records
Saved ptbxl_batch_0.npy  shape (1000, 2920, 12)
Processed 2000/21799 PTB-XL records
Saved ptbxl_batch_1.npy  shape (1000, 2920, 12)
Processed 3000/21799 PTB-XL records
Saved ptbxl_batch_2.npy  shape (1000, 2920, 12)
Processed 4000/21799 PTB-XL records
Saved ptbxl_batch_3.npy  shape (1000, 2920, 12)
Processed 5000/21799 PTB-XL records
Saved ptbxl_batch_4.npy  shape (1000, 2920, 12)
Processed 6000/21799 PTB-XL records
Saved ptbxl_batch_5.npy  shape (1000, 2920, 12)
Processed 7000/21799 PTB-XL records
Saved ptbxl_batch_6.npy  shape (1000, 2920, 12)
Processed 8000/21799 PTB-XL records
Saved ptbxl_batch_7.npy  shape (1000, 2920, 12)
Processed 9000/21799 PTB-XL records
Saved ptbxl_batch_8.npy  shape (1000, 2920, 12)
Processed 10000/21799 PTB-XL records
Saved ptbxl_batch_9.npy  shape (1000, 2920, 12)
Processed 11000/21799 PTB-XL records
Saved ptbxl_batch_10.npy  shape (1000, 2920, 12)
Processed 12000/21799 PTB-XL records
Saved ptbxl_batch_11.npy  shape (100

### Combine external test set

In [24]:
# Load processed positive and negative arrays
pos_data = np.load('samitrop_proc.npz')
neg_data = np.load('ptbxl_proc.npz')

# Combine the two datasets into a single external test set
X_test = np.concatenate([pos_data['X'], neg_data['X']])
y_test = np.concatenate([pos_data['y'], neg_data['y']])

print(f'External test shape: {X_test.shape}, positives: {y_test.sum()}')

# Save the combined external test set
np.savez_compressed('test_external.npz', X=X_test, y=y_test)
print('Saved test_external.npz')

External test shape: (23430, 2920, 12), positives: 1631
Saved test_external.npz
